In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta


In [14]:

pedido_post = '''[
    {
        "codigo": "ASARECHRB",
        "cotacao": 1000,
        "disponivel": true,
        "estoque": 6,
        "estoque_minimo": 5,
        "id": "623449824",
        "marca": "Recheados Brasil",
        "nome": "ASINHA RECHEADA",
        "quantidade": 0,
        "slug": "Asinha",
        "unidade": "un",
        "valorfinal": 0,
        "situacao": "Acima do estoque em 1.00 un⠀⠀"
    },
    {
        "codigo": "QUEIJCOA",
        "cotacao": 5,
        "disponivel": false,
        "estoque": 0,
        "estoque_minimo": 3,
        "id": "623450598",
        "marca": "Porcão",
        "nome": "QUEIJO COALHO",
        "quantidade": 0,
        "slug": "queijo",
        "unidade": "un",
        "valorfinal": 0,
        "situacao": "Abaixo do estoque.⠀⠀⠀⠀⠀⠀⠀⠀⠀"
    }
]'''

pedido_post = pd.read_json(pedido_post).to_dict(orient='records')
pedido_post


[{'codigo': 'ASARECHRB',
  'cotacao': 1000,
  'disponivel': True,
  'estoque': 6,
  'estoque_minimo': 5,
  'id': 623449824,
  'marca': 'Recheados Brasil',
  'nome': 'ASINHA RECHEADA',
  'quantidade': 0,
  'slug': 'Asinha',
  'unidade': 'un',
  'valorfinal': 0,
  'situacao': 'Acima do estoque em 1.00 un⠀⠀'},
 {'codigo': 'QUEIJCOA',
  'cotacao': 5,
  'disponivel': False,
  'estoque': 0,
  'estoque_minimo': 3,
  'id': 623450598,
  'marca': 'Porcão',
  'nome': 'QUEIJO COALHO',
  'quantidade': 0,
  'slug': 'queijo',
  'unidade': 'un',
  'valorfinal': 0,
  'situacao': 'Abaixo do estoque.⠀⠀⠀⠀⠀⠀⠀⠀⠀'}]

In [15]:

# variÃ¡veis
two_months_ago = datetime.today() - timedelta(weeks=4)


In [19]:
# ## Pedido
# pedido
pedido = pd.DataFrame(pedido_post)
pedido['valorfinal'] = pd.to_numeric(pedido['valorfinal'])
pedido = pd.pivot_table(pedido, values=['valorfinal'], index='marca', aggfunc='sum').reset_index()
pedido.columns = ['marca', 'total']


,codigo,cotacao,disponivel,estoque,estoque_minimo,id,marca,nome,quantidade,slug,unidade,valorfinal,situacao
0,ASARECHRB,1000,True,6,5,623449824,Recheados Brasil,ASINHA RECHEADA,0,Asinha,un,0,Acima do estoque em 1.00 un⠀⠀
1,QUEIJCOA,5,False,0,3,623450598,Porcão,QUEIJO COALHO,0,queijo,un,0,Abaixo do estoque.⠀⠀⠀⠀⠀⠀⠀⠀⠀


In [ ]:
# prazos
prazos = pd.read_csv('./prazos.csv')
prazos.columns = ['marca', 1, 2, 3]
prazos['n_parcelas'] = 3 - prazos.T.isna().sum()
prazos = prazos.merge(pedido)


: 

In [ ]:
# agenda
agenda = prazos.copy()
agenda
for parcela in [1, 2, 3]:
    agenda[parcela] = [datetime.today() + timedelta(days=x) if x >
                        0 else None for x in prazos[parcela]]
agenda['valor'] = agenda['total']/agenda['n_parcelas']
agenda = agenda.melt(id_vars=['marca', 'valor'], value_vars=[
                        1, 2, 3], value_name='data', var_name='parcela').dropna()
agenda['semana'] = [x.isocalendar()[1] for x in agenda['data']]


: 

In [ ]:
# peso
peso = agenda.copy()
peso = pd.pivot_table(
    agenda, index=peso['data'], aggfunc='sum').reset_index()


: 

In [ ]:
# ## SituaÃ§Ã£o
# contas
vendas = list()
data = {
    'formato': 'json',
    'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
    'dataInicial': two_months_ago.strftime('%d/%m/%Y'),
    'dataFinal': datetime.today().strftime('%d/%m/%Y'),
    'pagina': 1
    }
while True:
    try:
        url = 'https://api.tiny.com.br/api2/pedidos.pesquisa.php'
        vendas += [x['pedido']
                    for x in requests.post(url, data).json()['retorno']['pedidos']]
        data['pagina'] += 1
    except:
        break
vendas = pd.DataFrame(vendas)


: 

In [ ]:
# faturamento diÃ¡rio
faturamento_estimado = vendas['valor'].sum()/30


: 

In [ ]:
# contas
contas = list()
data = {
    'formato': 'json',
    'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
    'data_ini_vencimento': datetime.today().strftime('%d/%m/%Y'),
    'data_fim_vencimento': agenda['data'].max().strftime('%d/%m/%Y'),
    'pagina': 1
    }
while True:
    try:
        url = 'https://api.tiny.com.br/api2/contas.pagar.pesquisa.php'
        contas += [x['conta']
                    for x in requests.post(url, data).json()['retorno']['contas']]
        data['pagina'] += 1
    except:
        break
contas = pd.DataFrame(contas)
contas = contas.T.reindex(['data_vencimento', 'valor']).T
contas.columns = ['data', 'valor']
contas['data'] = [datetime.strptime(x, '%d/%m/%Y') for x in contas['data']]


: 

In [ ]:
# saldo
saldo = pd.DataFrame(contas['data'])
saldo['valor'] = np.repeat(faturamento_estimado, len(
    contas)) - pd.to_numeric(contas['valor'])


: 

In [ ]:
# ## DivisÃ£o por semanas
# saldo
saldo_semanal = saldo.copy()
saldo_semanal['semana'] = [x.isocalendar()[1] for x in saldo['data']]
saldo_semanal = pd.pivot_table(
    saldo_semanal, index='semana', values='valor', aggfunc='sum')


: 

In [ ]:
# compras
peso_semanal = peso.copy()
peso_semanal['semana'] = [x.isocalendar()[1] for x in peso['data']]
peso_semanal = pd.pivot_table(
    peso_semanal, index='semana', values='valor', aggfunc='sum')


: 

In [ ]:
# ## resultado
# analise
def rweek(semana):
    return datetime.fromisocalendar(datetime.today().year, semana, 1), datetime.fromisocalendar(datetime.today().year, semana, 7)
analise = peso_semanal.join(saldo_semanal, lsuffix='_compras', rsuffix='_saldo').fillna(faturamento_estimado*7)
analise['subtracao'] = analise['valor_saldo'] - analise['valor_compras']
analise['intervalo'] = [rweek(x) for x in analise.index]
analise['aprovado'] = [x > 0 for x in analise['subtracao']]
analise.reset_index(inplace=True)
analise = analise.T.reindex(
    ['aprovado', 'intervalo', 'subtracao', 'valor_compras', 'valor_saldo', 'semana']).T


: 

In [ ]:
# negadas
negadas = analise[analise['aprovado'] == False]
negadas = negadas.merge(agenda)
negadas = negadas.T.reindex(['marca', 'data', 'valor']).T


: 

In [ ]:
# ## Formatacao
negadas_f = negadas.copy()
negadas_f['data'] = [x.strftime('%d/%m/%Y') for x in negadas['data']]
negadas_f.to_dict(orient='records')
analise_f = analise.copy()
analise_f['intervalo'] = [[x[0].strftime(
    '%d/%m/%Y'), x[1].strftime('%d/%m/%Y')] for x in analise_f['intervalo']]
resultado = {'analise': analise_f.to_dict(orient='records'), 'negadas': negadas_f.to_dict(
    orient='records'), 'valor': negadas['valor'].sum()}
return resultado


: 